In [ ]:
import numpy as np
import sympy as sy
import matplotlib.pyplot as plt
import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app
from tqdm import tqdm
import random

In [ ]:
# Definiendo el toy model

n_particles = 1\n",
system = mm.System()\n",
mass   = 14.007 * unit.amu # masa del átomo N\n",
system.addParticle(mass)

force = mm.CustomExternalForce('Eo*(C*x^4-x^2)+M*x + (K/2.0)*(y^2+z^2)')
force.addGlobalParameter('K', 35 * unit.kilocalories_per_mole/unit.angstrom**2)
force.addGlobalParameter('Eo', 6.0 * unit.kilocalories_per_mole/unit.angstrom**2)
force.addGlobalParameter('C', 0.4 * 1/unit.angstrom**2)
force.addGlobalParameter('M', 0.25 * unit.kilocalories_per_mole/unit.angstrom)
force.addParticle(0, [])
system.addForce(force)

In [ ]:
 "# Intento de 4 réplicas\n",
    "\n",
    "kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA\n",
    "temperature = 300*unit.kelvin\n",
    "pressure    = None\n",
    "\n",
    "step_size       = 0.01*unit.picoseconds\n",
    "num_steps       = 20000\n",
    "saving_period   = 100\n",
    "num_steps_saved = int(num_steps/saving_period)\n",
    "\n",
    "friction   = 1.0/unit.picosecond\n",
    "integrator = mm.LangevinIntegrator(temperature, friction, step_size)\n",
    "\n",
    "platform_name = 'CPU'  \n",
    "platform = mm.Platform.getPlatformByName(platform_name)\n",
    "\n",
    "times      = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.picoseconds)\n",
    "positions  = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms)\n",
    "velocities = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms/unit.picosecond)\n",
    "potential_energies   = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)\n",
    "kinetic_energies    = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)\n",
    "\n",
    "initial_positions  = np.zeros([n_particles, 3], np.float32) * unit.nanometers\n",
    "\n",
    "#initial_velocities = None # Las velocidades serán adjudicadas aleatoriamente según la distribución Maxwell-Boltzmann del estado termodinámico\n",
    "\n",
    "context = mm.Context(system, integrator, platform)\n",
    "context.setPositions(initial_positions)\n",
    "context.setVelocitiesToTemperature(temperature)\n",
    "\n",
    "state = context.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "times[0] = state.getTime()\n",
    "positions[0] = state.getPositions()[0]\n",
    "velocities[0] = state.getVelocities()[0]\n",
    "kinetic_energies[0]=state.getKineticEnergy()\n",
    "potential_energies[0]=state.getPotentialEnergy()\n",
    "\n",
    "for ii in tqdm(range(num_steps_saved)):\n",
    "    context.getIntegrator().step(saving_period)\n",
    "    state = context.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "    times[ii] = state.getTime()\n",
    "    positions[ii] = state.getPositions()[0]\n",
    "    velocities[ii] = state.getVelocities()[0]\n",
    "    kinetic_energies[ii]=state.getKineticEnergy()\n",
    "    potential_energies[ii]=state.getPotentialEnergy()\n",
    "    \n",
    "\n",
    "temperature2 = 305*unit.kelvin\n",
    "step_size2       = 0.01*unit.picoseconds\n",
    "num_steps2       = 20000\n",
    "saving_period2   = 100\n",
    "num_steps_saved2 = int(num_steps/saving_period)\n",
    "\n",
    "friction2   = 1.0/unit.picosecond\n",
    "integrator2 = mm.LangevinIntegrator(temperature2, friction2, step_size2)\n",
    "\n",
    "times2      = unit.Quantity(np.zeros([num_steps_saved2], np.float32), unit.picoseconds)\n",
    "positions2  = unit.Quantity(np.zeros([num_steps_saved2,3], np.float32), unit.angstroms)\n",
    "velocities2 = unit.Quantity(np.zeros([num_steps_saved2,3], np.float32), unit.angstroms/unit.picosecond)\n",
    "potential_energies2   = unit.Quantity(np.zeros([num_steps_saved2], np.float32), unit.kilocalories_per_mole)\n",
    "kinetic_energies2     = unit.Quantity(np.zeros([num_steps_saved2], np.float32), unit.kilocalories_per_mole)\n",
    "\n",
    "initial_positions2  = np.zeros([n_particles, 3], np.float32) * unit.nanometers\n",
    "\n",
    "#initial_velocities = None # Las velocidades serán adjudicadas aleatoriamente según la distribución Maxwell-Boltzmann del estado termodinámico\n",
    "\n",
    "context2 = mm.Context(system, integrator2, platform)\n",
    "context2.setPositions(initial_positions2)\n",
    "context2.setVelocitiesToTemperature(temperature2)\n",
    "\n",
    "state2 = context2.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "times2[0] = state2.getTime()\n",
    "positions2[0] = state2.getPositions()[0]\n",
    "velocities2[0] = state2.getVelocities()[0]\n",
    "kinetic_energies2[0]=state2.getKineticEnergy()\n",
    "potential_energies2[0]=state2.getPotentialEnergy()\n",
    "\n",
    "for jj in tqdm(range(num_steps_saved)):\n",
    "    context2.getIntegrator().step(saving_period)\n",
    "    state2 = context2.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "    times2[jj] = state2.getTime()\n",
    "    positions2[jj] = state2.getPositions()[0]\n",
    "    velocities2[jj] = state2.getVelocities()[0]\n",
    "    kinetic_energies2[jj]=state2.getKineticEnergy()\n",
    "    potential_energies[jj]=state2.getPotentialEnergy()\n",
    "    \n",
    "temperature3 = 310*unit.kelvin\n",
    "pressure    = None\n",
    "\n",
    "step_size3       = 0.01*unit.picoseconds\n",
    "num_steps3       = 20000\n",
    "saving_period3   = 100\n",
    "num_steps_saved3 = int(num_steps/saving_period)\n",
    "\n",
    "friction3   = 1.0/unit.picosecond\n",
    "integrator3 = mm.LangevinIntegrator(temperature3, friction3, step_size3)\n",
    "\n",
    "platform_name = 'CPU'  \n",
    "platform = mm.Platform.getPlatformByName(platform_name)\n",
    "\n",
    "times3      = unit.Quantity(np.zeros([num_steps_saved3], np.float32), unit.picoseconds)\n",
    "positions3  = unit.Quantity(np.zeros([num_steps_saved3,3], np.float32), unit.angstroms)\n",
    "velocities3 = unit.Quantity(np.zeros([num_steps_saved3,3], np.float32), unit.angstroms/unit.picosecond)\n",
    "potential_energies3   = unit.Quantity(np.zeros([num_steps_saved3], np.float32), unit.kilocalories_per_mole)\n",
    "kinetic_energies3    = unit.Quantity(np.zeros([num_steps_saved3], np.float32), unit.kilocalories_per_mole)\n",
    "\n",
    "initial_positions3  = np.zeros([n_particles, 3], np.float32) * unit.nanometers\n",
    "\n",
    "#initial_velocities = None # Las velocidades serán adjudicadas aleatoriamente según la distribución Maxwell-Boltzmann del estado termodinámico\n",
    "\n",
    "context3 = mm.Context(system, integrator3, platform)\n",
    "context3.setPositions(initial_positions3)\n",
    "context3.setVelocitiesToTemperature(temperature3)\n",
    "\n",
    "state3 = context.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "times3[0] = state3.getTime()\n",
    "positions3[0] = state3.getPositions()[0]\n",
    "velocities3[0] = state3.getVelocities()[0]\n",
    "kinetic_energies3[0]=state3.getKineticEnergy()\n",
    "potential_energies3[0]=state3.getPotentialEnergy()\n",
    "\n",
    "for ii in tqdm(range(num_steps_saved)):\n",
    "    context3.getIntegrator().step(saving_period)\n",
    "    state3 = context3.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "    times3[ii] = state3.getTime()\n",
    "    positions3[ii] = state3.getPositions()[0]\n",
    "    velocities3[ii] = state3.getVelocities()[0]\n",
    "    kinetic_energies3[ii]=state3.getKineticEnergy()\n",
    "    potential_energies3[ii]=state3.getPotentialEnergy()\n",
    "    \n",
    "\n",
    "temperature4 = 315*unit.kelvin\n",
    "integrator4 = mm.LangevinIntegrator(temperature4, friction, step_size)\n",
    "\n",
    "times4      = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.picoseconds)\n",
    "positions4  = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms)\n",
    "velocities4 = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms/unit.picosecond)\n",
    "potential_energies4   = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)\n",
    "kinetic_energies4     = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)\n",
    "\n",
    "initial_positions4  = np.zeros([n_particles, 3], np.float32) * unit.nanometers\n",
    "\n",
    "#initial_velocities = None # Las velocidades serán adjudicadas aleatoriamente según la distribución Maxwell-Boltzmann del estado termodinámico\n",
    "\n",
    "context4 = mm.Context(system, integrator4, platform)\n",
    "context4.setPositions(initial_positions4)\n",
    "context4.setVelocitiesToTemperature(temperature4)\n",
    "\n",
    "state4 = context4.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "times4[0] = state4.getTime()\n",
    "positions4[0] = state4.getPositions()[0]\n",
    "velocities4[0] = state4.getVelocities()[0]\n",
    "kinetic_energies4[0]=state4.getKineticEnergy()\n",
    "potential_energies4[0]=state4.getPotentialEnergy()\n",
    "\n",
    "for jj in tqdm(range(num_steps_saved)):\n",
    "    context4.getIntegrator().step(saving_period)\n",
    "    state4 = context4.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "    times4[jj] = state4.getTime()\n",
    "    positions4[jj] = state4.getPositions()[0]\n",
    "    velocities4[jj] = state4.getVelocities()[0]\n",
    "    kinetic_energies4[jj]=state4.getKineticEnergy()\n",
    "    potential_energies4[jj]=state4.getPotentialEnergy()"

In [ ]:
"# Analizando las reṕlicas\n",
    "kBT1 = kB * temperature\n",
    "kBT2 = kB * temperature2\n",
    "kBT3 = kB * temperature3\n",
    "kBT4 = kB * temperature4\n",
    "\n",
    "delta = (1.0/kBT2 -1.0/kBT1)*(potential_energies[-1]-potential_energies2[-1])\n",
    "delta2 = (1.0/kBT4 -1.0/kBT3)*(potential_energies3[-1]-potential_energies4[-1])\n",
    "\n",
    "if delta <= 0:\n",
    "    P1 = 1\n",
    "elif delta > 0:\n",
    "    Ex = np.exp(-(delta))\n",
    "    rnd=random.random()\n",
    "    if rnd <= Ex:\n",
    "        P1 = 1\n",
    "        timesc1      = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.picoseconds)\n",
    "        positionsc1  = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms)\n",
    "        velocitiesc1 = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms/unit.picosecond)\n",
    "        potential_energiesc1 = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)\n",
    "        kinetic_energiesc1 = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)\n",
    "\n",
    "        initial_positions  = positions2[-1] \n",
    "\n",
    "        initial_velocities = velocities2[-1]\n",
    "\n",
    "        context = mm.Context(system, integrator, platform)\n",
    "        context.setPositions(initial_positions)\n",
    "        context.setVelocitiesToTemperature(temperature2)\n",
    "\n",
    "        state = context.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "        timesc1[ii] = state.getTime()\n",
    "        positionsc1[ii] = state.getPositions()[0]\n",
    "        velocitiesc1[ii] = state.getVelocities()[0]\n",
    "        kinetic_energiesc1[ii]=state.getKineticEnergy()\n",
    "        potential_energiesc1[ii]=state.getPotentialEnergy()\n",
    "\n",
    "        for ii in tqdm(range(num_steps_saved)):\n",
    "            context.getIntegrator().step(saving_period)\n",
    "            state = context.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "            timesc1[ii] = state.getTime()\n",
    "            positionsc1[ii] = state.getPositions()[0]\n",
    "            velocitiesc1[ii] = state.getVelocities()[0]\n",
    "            kinetic_energiesc1[ii]=state.getKineticEnergy()\n",
    "            potential_energiesc1[ii]=state.getPotentialEnergy()\n",
    "\n",
    "        times      = np.append(times,timesc1)* unit.picoseconds\n",
    "        positions  = np.append(positions,positionsc1, axis =0)* unit.angstroms\n",
    "        velocities = np.append(velocities,velocitiesc1, axis =0) * unit.angstroms/unit.picosecond\n",
    "        potential_energies = np.append(potential_energies,potential_energiesc1, axis =0)* unit.kilocalories_per_mole\n",
    "        kinetic_energies = np.append(kinetic_energies, kinetic_energiesc1, axis =0) * unit.kilocalorie_per_mole\n",
    "\n",
    "        initial_positions2  = positions[-1]\n",
    "\n",
    "        initial_velocities2 = velocities[-1]\n",
    "        context2 = mm.Context(system, integrator2, platform)\n",
    "        context2.setPositions(initial_positions)\n",
    "        context2.setVelocitiesToTemperature(temperature)\n",
    "\n",
    "        state2 = context2.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "        timesc2[ii] = state2.getTime()\n",
    "        positionsc2[ii] = state2.getPositions()[0]\n",
    "        velocitiesc2[ii] = state2.getVelocities()[0]\n",
    "        kinetic_energiesc2[ii]=state2.getKineticEnergy()\n",
    "        potential_energiesc2[ii]=state2.getPotentialEnergy()\n",
    "\n",
    "        for jj in tqdm(range(num_steps_saved)):\n",
    "            context2.getIntegrator().step(saving_period)\n",
    "            state2 = context2.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "            timesc2[jj] = state2.getTime()\n",
    "            positionsc2[jj] = state2.getPositions()[0]\n",
    "            velocitiesc2[jj] = state2.getVelocities()[0]\n",
    "            kinetic_energiesc2[jj]=state2.getKineticEnergy()\n",
    "            potential_energiesc2[jj]=state2.getPotentialEnergy()\n",
    "\n",
    "        times2      = np.append(times,timesc2)* unit.picoseconds\n",
    "        positions2  = np.append(positions,positionsc2, axis =0)* unit.angstroms\n",
    "        velocities2 = np.append(velocities,velocitiesc2, axis =0) * unit.angstroms/unit.picosecond\n",
    "        potential_energies2 = np.append(potential_energies,potential_energiesc2, axis =0)* unit.kilocalories_per_mole\n",
    "        kinetic_energies2 = np.append(kinetic_energies, kinetic_energiesc2, axis =0) * unit.kilocalorie_per_mole\n",
    "\n",
    "    else:\n",
    "        P1 = 0\n",
    "        print(\"R1 y R2 no se intercambian\")\n",
    "\n",
    "if delta2 <= 0:\n",
    "    P2 = 1\n",
    "elif delta2 > 0:\n",
    "    Ex2 = np.exp(-(delta2))\n",
    "    rnd=random.random()\n",
    "    if rnd <= Ex2:\n",
    "        P2 = 1\n",
    "        timesc3      = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.picoseconds)\n",
    "        positionsc3  = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms)\n",
    "        velocitiesc3 = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms/unit.picosecond)\n",
    "        potential_energiesc3 = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)\n",
    "        kinetic_energiesc3 = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)\n",
    "\n",
    "        initial_positions3  = positions4[-1] \n",
    "\n",
    "        initial_velocities3 = velocities4[-1]\n",
    "\n",
    "        context3 = mm.Context(system, integrator, platform)\n",
    "        context.setPositions(initial_positions3)\n",
    "        context.setVelocitiesToTemperature(temperature4)\n",
    "\n",
    "        state3 = context3.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "        timesc3[ii] = state3.getTime()\n",
    "        positionsc3[ii] = state3.getPositions()[0]\n",
    "        velocitiesc3[ii] = state3.getVelocities()[0]\n",
    "        kinetic_energiesc3[ii]=state3.getKineticEnergy()\n",
    "        potential_energiesc3[ii]=state3.getPotentialEnergy()\n",
    "\n",
    "        for ii in tqdm(range(num_steps_saved)):\n",
    "            context3.getIntegrator().step(saving_period)\n",
    "            state3 = context3.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "            timesc3[ii] = state3.getTime()\n",
    "            positionsc3[ii] = state3.getPositions()[0]\n",
    "            velocitiesc3[ii] = state3.getVelocities()[0]\n",
    "            kinetic_energiesc3[ii]=state3.getKineticEnergy()\n",
    "            potential_energiesc3[ii]=state3.getPotentialEnergy()\n",
    "\n",
    "        times3      = np.append(times,timesc3)* unit.picoseconds\n",
    "        positions3  = np.append(positions,positionsc3, axis =0)* unit.angstroms\n",
    "        velocities3 = np.append(velocities,velocitiesc3, axis =0) * unit.angstroms/unit.picosecond\n",
    "        potential_energies3 = np.append(potential_energies,potential_energiesc3, axis =0)* unit.kilocalories_per_mole\n",
    "        kinetic_energies3 = np.append(kinetic_energies, kinetic_energiesc3, axis =0) * unit.kilocalorie_per_mole\n",
    "\n",
    "        initial_positions4  = positions3[-1]\n",
    "\n",
    "        initial_velocities4 = velocities3[-1]\n",
    "        context4 = mm.Context(system, integrator2, platform)\n",
    "        context4.setPositions(initial_positions)\n",
    "        context4.setVelocitiesToTemperature(temperature3)\n",
    "\n",
    "        state4 = context4.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "        timesc4[ii] = state4.getTime()\n",
    "        positionsc4[ii] = state4.getPositions()[0]\n",
    "        velocitiesc4[ii] = state4.getVelocities()[0]\n",
    "        kinetic_energiesc4[ii]=state4.getKineticEnergy()\n",
    "        potential_energiesc4[ii]=state4.getPotentialEnergy()\n",
    "\n",
    "        for jj in tqdm(range(num_steps_saved)):\n",
    "            context4.getIntegrator().step(saving_period)\n",
    "            state4 = context4.getState(getEnergy=True, getPositions=True, getVelocities=True)\n",
    "            timesc4[jj] = state4.getTime()\n",
    "            positionsc4[jj] = state4.getPositions()[0]\n",
    "            velocitiesc4[jj] = state4.getVelocities()[0]\n",
    "            kinetic_energiesc4[jj]=state4.getKineticEnergy()\n",
    "            potential_energiesc4[jj]=state4.getPotentialEnergy()\n",
    "\n",
    "        times4      = np.append(times,timesc4)* unit.picoseconds\n",
    "        positions4  = np.append(positions,positionsc4, axis =0)* unit.angstroms\n",
    "        velocities4 = np.append(velocities,velocitiesc4, axis =0) * unit.angstroms/unit.picosecond\n",
    "        potential_energies4 = np.append(potential_energies,potential_energiesc4, axis =0)* unit.kilocalories_per_mole\n",
    "        kinetic_energies4 = np.append(kinetic_energies, kinetic_energiesc4, axis =0) * unit.kilocalorie_per_mole\n",
    "\n",
    "    else:\n",
    "        P2 = 0\n",
    "        print(\"R3 y R4 no se intercambian\")